In [2]:
!pip install ultralytics

In [1]:
!pip install fiftyone

  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
     ---------------------------------------- 0.0/65.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/65.9 MB 3.5 MB/s eta 0:00:19
     ---------------------------------------- 0.3/65.9 MB 4.1 MB/s eta 0:00:16
     ---------------------------------------- 0.5/65.9 MB 4.0 MB/s eta 0:00:17
     ---------------------------------------- 0.7/65.9 MB 4.0 MB/s eta 0:00:17
      --------------------------------------- 0.9/65.9 MB 4.2 MB/s eta 0:00:16
      --------------------------------------- 1.0/65.9 MB 4.1 MB/s eta 0:00:16
      --------------------------------------- 1.2/65.9 MB 4.2 MB/s eta 0:00:16
      --------------------------------------- 1.5/65.9 MB 4.3 MB/s eta 0:00:16
     - -------------------------------------- 1.6/65.9 MB 4.0 MB/s eta 0:00:16
     - -------------------------------------- 1.8/65.9 MB 4.2 MB/s eta 0:00:16
     - -------------------------------------- 2.0/65.9 MB 4.1 MB/s eta 0

In [3]:
import fiftyone.zoo as foz
import fiftyone as fo
from fiftyone import ViewField as F
from ultralytics import YOLO
import os
from PIL import Image
import shutil
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from sklearn.metrics import precision_score, recall_score

Migrating database to v0.23.3


In [4]:
typesDataset =["test", "train", "validation"]

for typeDataset in typesDataset:
    # Load the dataset
    dataset = foz.load_zoo_dataset(
        "open-images-v7",
        split=typeDataset,
        label_types=["detections"],
        classes=["Vehicle registration plate"],
        max_samples=200000,
    )

Only found 2065 (<200000) samples matching your requirements
 100% |█████████████████| 2065/2065 [9.4m elapsed, 0s remaining, 3.5 files/s]       
Dataset info written to 'C:\Users\alluv\fiftyone\open-images-v7\info.json'
Loading 'open-images-v7' split 'test'
 100% |███████████████| 2065/2065 [11.0s elapsed, 0s remaining, 174.2 samples/s]      
Dataset 'open-images-v7-test-200000' created
 100% |██████|    4.8Gb/4.8Gb [2.9m elapsed, 0s remaining, 30.3Mb/s]      


OSError: [Errno 28] No space left on device

In [ ]:
# List all datasets
datasets = fo.list_datasets()

# Print the datasets
for dataset_name in datasets:
    print(dataset_name)

In [ ]:
# List of datasets
dataset_names = [
    "open-images-v7-test-200000",
    "open-images-v7-train-200000",
    "open-images-v7-validation-200000"
]

# Iteratively load and print the number of samples for each dataset
for dataset_name in dataset_names:
    dataset = fo.load_dataset(dataset_name)
    print(f"Number of samples in {dataset_name}: {len(dataset)}")

In [ ]:
dataset = fo.load_dataset('open-images-v7-train-200000')

# Create a session
session = fo.launch_app(dataset)

# Use the session...

# When done, close the session
session.close()

In [ ]:
# List of original datasets and their corresponding filtered dataset names
datasets_info = {
    "open-images-v7-test-200000": "FilteredDatasetTest",
    "open-images-v7-train-200000": "FilteredDatasetTrain",
    "open-images-v7-validation-200000": "FilteredDatasetValidation"
}

# Iteratively process and create new datasets for each original dataset
for orig_dataset_name, filtered_dataset_name in datasets_info.items():
    
    # Load the original dataset
    dataset = fo.load_dataset(orig_dataset_name)
    
    # Define a view that only includes samples whose `detections` field only contains `Mushroom` labels
    only_mushrooms_view = dataset.match(
        {"$expr": {
            "$eq": [{"$setDifference": ["$ground_truth.detections.label", ["Vehicle registration plate"]]}, []]
        }}
    )
    
    # Delete the filtered dataset if it exists
    if fo.dataset_exists(filtered_dataset_name):
        fo.delete_dataset(filtered_dataset_name)
    
    # Create a new dataset with the appropriate name
    filtered_dataset = fo.Dataset(name=filtered_dataset_name)
    
    # Copy samples from view to new dataset
    for sample in only_mushrooms_view:
        filtered_dataset.add_sample(sample)

# Delete the original datasets
for orig_dataset_name in datasets_info.keys():
    if fo.dataset_exists(orig_dataset_name):
        fo.delete_dataset(orig_dataset_name)


In [ ]:
# List all datasets
datasets = fo.list_datasets()

# Print the datasets
for dataset_name in datasets:
    print(dataset_name)

In [ ]:
# Load the dataset
dataset_train = fo.load_dataset("FilteredDatasetTrain")
dataset_validation = fo.load_dataset("FilteredDatasetValidation")
dataset_test = fo.load_dataset("FilteredDatasetTest")

# Get the class names in the dataset
class_names = dataset_train.distinct("ground_truth.detections.label")
class_names.sort()
class_dict = {name: idx for idx, name in enumerate(class_names)}

# Output directories for YOLO formatted labels and images
output_dir_imgs_train = "./data/images/train/"
output_dir_lbls_train = "./data/labels/train/"
output_dir_imgs_val = "./data/images/val/"
output_dir_lbls_val = "./data/labels/val/"
output_dir_imgs_test = "./data/images/test/"
output_dir_lbls_test = "./data/labels/test/"

# Ensure directories exist
os.makedirs(output_dir_imgs_train, exist_ok=True)
os.makedirs(output_dir_lbls_train, exist_ok=True)
os.makedirs(output_dir_imgs_val, exist_ok=True)
os.makedirs(output_dir_lbls_val, exist_ok=True)
os.makedirs(output_dir_imgs_test, exist_ok=True)
os.makedirs(output_dir_lbls_test, exist_ok=True)

datasets = [
    (dataset_train, output_dir_imgs_train, output_dir_lbls_train),
    (dataset_validation, output_dir_imgs_val, output_dir_lbls_val),
    (dataset_test, output_dir_imgs_test, output_dir_lbls_test),
]

for dataset, output_dir_imgs, output_dir_lbls in datasets:
    for sample in dataset:
        # Create a file for each image
        filename = os.path.splitext(os.path.basename(sample.filepath))[0]
        with open(os.path.join(output_dir_lbls, filename + ".txt"), "w") as f:
            # Convert each detection in the sample
            for detection in sample.ground_truth.detections:
                # Convert the bounding box format
                # From [xmin, ymin, width, height] (in relative coordinates)
                # To [class, xcenter, ycenter, width, height] (in relative coordinates)
                bbox = detection.bounding_box
                xcenter = bbox[0] + bbox[2] / 2
                ycenter = bbox[1] + bbox[3] / 2
                class_idx = class_dict[detection.label]

                # Write the bounding box to the file
                f.write(f"{class_idx} {xcenter} {ycenter} {bbox[2]} {bbox[3]}\n")
    
        # Copy the corresponding image file to the output directory
        shutil.copy(sample.filepath, os.path.join(output_dir_imgs, filename + ".jpg"))


In [ ]:

# Path to images and labels
image_dir = "data/images/train/"
label_dir = "data/labels/train/"

# Get all image files from the directory
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# Select 3 sample images
selected_image_files = image_files[:3]

# For each selected image file
for image_file in selected_image_files:
    # Read image
    img = cv2.imread(os.path.join(image_dir, image_file))

    if img is None:
        print(f"Failed to load image {image_file}")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert color space for matplotlib
    img_height, img_width = img.shape[:2]

    # Corresponding label file
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(label_dir, label_file)

    # If label file exists
    if os.path.exists(label_path):
        with open(label_path, "r") as file:
            lines = file.readlines()

        # For each line in label file (each object)
        for line in lines:
            class_idx, x, y, w, h = map(float, line.split())
            x *= img_width
            w *= img_width
            y *= img_height
            h *= img_height

            # Draw bounding box
            start_point = (int(x - w/2), int(y - h/2))  # top left corner
            end_point = (int(x + w/2), int(y + h/2))  # bottom right corner
            color = (0, 255, 0)  # green color in RGB
            thickness = 2
            img = cv2.rectangle(img, start_point, end_point, color, thickness)

        # Display image with bounding boxes
        plt.imshow(img)
        plt.title(image_file)
        plt.show()
    else:
        print(f"No label file found for {image_file}")
